## Model Training

#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

#### Import the CSV Data as Pandas DataFrame

In [2]:
df_=pd.read_csv('data/stud.csv')
df=df_.copy()

#### Show Top 5 Records

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Preparing X and Y variables

In [4]:
X=df.drop(columns=["math_score"],axis=1)
y=df["math_score"]

In [5]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [6]:
X = preprocessor.fit_transform(X)

In [7]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

### Creating an evaluation function

In [8]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2=r2_score(true,predicted)
    return mae,mse,rmse,r2

In [17]:
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "KNN": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "SVR": SVR(),
    "CatBoost": CatBoostRegressor(verbose=0),
    "XGBoost": XGBRegressor()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    # Make preditcions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    # Evaluate model
    mae_train, mse_train, rmse_train, r2_train = evaluate_model(y_train, y_train_pred)
    mae_test, mse_test, rmse_test, r2_test = evaluate_model(y_test, y_test_pred)
    model_list.append(list(models.keys())[i])
    print(model.__class__.__name__)
    print("\n")
    print("Model performance on training set")
    print("MAE: {:.4f}".format(mae_train))
    print("MSE: {:.4f}".format(mse_train))
    print("RMSE: {:.4f}".format(rmse_train))
    print("R2: {:.4f}".format(r2_train))
    print("="*50)
    print("Model performance on test set")
    print("MAE: {:.4f}".format(mae_test))
    print("MSE: {:.4f}".format(mse_test))
    print("RMSE: {:.4f}".format(rmse_test))
    print("R2: {:.4f}".format(r2_test))
    r2_list.append(r2_test)
    print("="*100)
    

LinearRegression


Model performance on training set
MAE: 4.2715
MSE: 28.4013
RMSE: 5.3293
R2: 0.8740
Model performance on test set
MAE: 4.2222
MSE: 29.4326
RMSE: 5.4252
R2: 0.8790
Ridge


Model performance on training set
MAE: 4.2650
MSE: 28.3378
RMSE: 5.3233
R2: 0.8743
Model performance on test set
MAE: 4.2111
MSE: 29.0563
RMSE: 5.3904
R2: 0.8806
Lasso


Model performance on training set
MAE: 5.2063
MSE: 43.4784
RMSE: 6.5938
R2: 0.8071
Model performance on test set
MAE: 5.1579
MSE: 42.5064
RMSE: 6.5197
R2: 0.8253
KNeighborsRegressor


Model performance on training set
MAE: 4.5168
MSE: 32.5799
RMSE: 5.7079
R2: 0.8555
Model performance on test set
MAE: 5.6210
MSE: 52.6066
RMSE: 7.2530
R2: 0.7838
DecisionTreeRegressor


Model performance on training set
MAE: 0.0187
MSE: 0.0781
RMSE: 0.2795
R2: 0.9997
Model performance on test set
MAE: 6.4400
MSE: 66.2900
RMSE: 8.1419
R2: 0.7276
RandomForestRegressor


Model performance on training set
MAE: 1.8489
MSE: 5.3997
RMSE: 2.3237
R2: 0.9760
Mode

In [18]:
pd.DataFrame({"Model":model_list,"R2":r2_list}).sort_values(by="R2",ascending=False)

,Model,R2
1,Ridge,0.880593
0,LinearRegression,0.879046
6,AdaBoost,0.854865
8,CatBoost,0.851632
5,RandomForest,0.848793
9,XGBoost,0.827797
2,Lasso,0.825320
3,KNN,0.783813
7,SVR,0.728600
4,DecisionTree,0.727581


# Linear Regression

In [21]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score=r2_score(y_test, y_pred)*100
print("Accuracy of the model is: %.2f" % score)

Accuracy of the model is: 87.90
